# Import records from Google Sheets

The curation of metadata records is done in a [Google Sheets document](https://docs.google.com/spreadsheets/d/1qIBpnP93ywwY0Y0Ujz17XYgpB4T7etbSBlQtz43kaSQ/edit?usp=sharing) with four sheets (tabs), one per document type.

This notebook will read the current contents of the sheet into a Pandas dataframe. Only "completed" rows marked with a "V" are used.
The records are converted into Python dictionaries and then saved into JSONL files, split into separate train and test subsets.

In [1]:
# Read the metadata from the Google Sheets document into Pandas dataframes

import pandas as pd
import urllib.parse

DOC_ID = "1qIBpnP93ywwY0Y0Ujz17XYgpB4T7etbSBlQtz43kaSQ"  # Google Sheets id
LANGUAGES = ('fin', 'swe', 'eng')

SHEET_NAMES = {
    "thes": "Theses",
    "docthes": "Doctoral theses",
    "serial": "Serial publications",
    "mono": "Monographs"
}

def read_sheet(doc_id, sheet_name, sheet_id):
    csv_url = f"https://docs.google.com/spreadsheets/d/{doc_id}/gviz/tq?tqx=out:csv&sheet={urllib.parse.quote(sheet_name)}"
    df = pd.read_csv(csv_url, dtype=str, na_filter=False)
    # add a new "rowid" column, with values like "thes37", in between other housekeeping columns
    df["rowid"] = df.index + 2
    df["rowid"] = df["rowid"].apply(lambda x: f"{sheet_id}{x}")
    df.insert(3, "rowid", df.pop("rowid"))
    # restrict the dataframe to only the "completed" rows
    df = df.loc[df["Status"].str.startswith("V", na=False)]
    # restrict to monolingual records in the languages we are interested in
    df = df.loc[df["language/iso"].isin(LANGUAGES)]
    return df

sheets = {sheet_id: read_sheet(DOC_ID, sheet_name, sheet_id)
          for sheet_id, sheet_name in SHEET_NAMES.items()}

In [2]:
# Calculate language statistics for each sheet (document type) and combine into an overview

language_counts = {sheet_id: sheet["language/iso"].value_counts().rename(sheet_id).astype(int)
                   for sheet_id, sheet in sheets.items()}

langstat = pd.concat(language_counts.values(), axis=1).fillna(0).astype(int)
langstat['TOTAL'] = langstat.sum(axis=1)
langstat.loc['TOTAL'] = langstat.sum()
langstat

,thes,docthes,serial,mono,TOTAL
language/iso,,,,,
fin,89,54,74,77,294
eng,56,99,66,50,271
swe,54,25,59,23,161
TOTAL,199,178,199,150,726


In [3]:
# Calculate by-repository statistics for each sheet (document type) and combine into an overview

repo_counts = {sheet_id: sheet["Repository"].value_counts().rename(sheet_id).astype(int)
               for sheet_id, sheet in sheets.items()}

repostat = pd.concat(repo_counts.values(), axis=1).fillna(0).astype(int)
repostat['TOTAL'] = repostat.sum(axis=1)
repostat.loc['TOTAL'] = repostat.sum()
repostat

,thes,docthes,serial,mono,TOTAL
Repository,,,,,
Theseus,91,0,95,45,231
Doria,29,50,16,16,111
Osuva,25,23,14,10,72
Taju,23,3,16,35,77
Trepo,16,35,0,7,58
UtuPub,15,40,1,0,56
LutPub,0,27,2,0,29
Kaisu,0,0,42,2,44
Julkari,0,0,13,35,48


In [4]:
# Calculate by-COAR-type statistics for each sheet (document type) and combine into an overview

coar_counts = {sheet_id: sheet["type/coar"].value_counts().rename(sheet_id).astype(int)
               for sheet_id, sheet in sheets.items()}

coarstat = pd.concat(coar_counts.values(), axis=1).fillna(0).astype(int)
coarstat['TOTAL'] = coarstat.sum(axis=1)
coarstat.loc['TOTAL'] = coarstat.sum()
coarstat

,thes,docthes,serial,mono,TOTAL
type/coar,,,,,
master thesis,114,0,0,0,114
bachelor thesis,79,0,0,0,79
thesis,4,0,0,0,4
seminar thesis,2,0,0,0,2
doctoral thesis,0,178,0,0,178
research report,0,0,69,39,108
journal article,0,0,35,2,37
research article,0,0,31,7,38
newspaper article,0,0,19,0,19


In [5]:
DROP_COLS = ('Collection', 'Status')
MULTI_VALUE_COLS = {
    'contributor',
    'identifier/isbn',
    'publisher',
    'relation/isbn',
    'subject',
    'title/alternative'
}

# every 1 out of TRAIN_TEST_SPLIT_FACTOR records (on average) will be placed in test set
TRAIN_TEST_SPLIT_FACTOR = 4

import json
import re
import zlib

def is_valid_col(col):
    if col in DROP_COLS:
        return False
    if col.endswith('X'):
        return False
    if col.startswith('Unnamed'):
        return False
    return True

def url_to_id(url):
    return re.sub(r"(.+)/bitstream/handle/(\d+)/(\d+)/.*", r"\1/handle/\2/\3", url)

def is_multivalue_col(col):
    return col in MULTI_VALUE_COLS or col.split('/')[0] in MULTI_VALUE_COLS

def convert_colname(col):
    if col in ('rowid', 'url'):
        return col
    elif col == 'Repository':
        return 'repository'
    return 'dc.' + '.'.join(col.split('/'))

def to_dict(row):
    d = {}
    for col, val in row.items():
        val = val.strip()
        if col == 'url':
            d["id"] = url_to_id(val)
        if is_valid_col(col) and val != '':
            d[convert_colname(col)] = val.split("\n") if is_multivalue_col(col) else val
    return d

def is_test_record(rec):
    """deterministically select, based on the record ID, whether a record goes into the train or test set"""
    return 3 * zlib.crc32(rec['id'].encode('utf-8')) % TRAIN_TEST_SPLIT_FACTOR == 1

for sheet_id, sheet in sheets.items():
    for lang in LANGUAGES:
        df = sheet[sheet['language/iso'] == lang]
        records = [to_dict(row) for _, row in df.iterrows()]
        with (open(f"../metadata/{sheet_id}-{lang}-train.jsonl", "w") as trainfile,
              open(f"../metadata/{sheet_id}-{lang}-test.jsonl", "w") as testfile):
            ntrain = ntest = 0
            for rec in records:
                if is_test_record(rec):
                    outfile = testfile
                    subset = 'test'
                    ntest += 1
                else:
                    outfile = trainfile
                    subset = 'train'
                    ntrain += 1
                header = {'doctype': sheet_id, 'subset': subset}
                json.dump(header | rec, outfile)
                outfile.write("\n")
        print(f"{sheet_id}-{lang}:\twrote {len(records)} records ({ntrain} train, {ntest} test ({100*ntest/(ntrain+ntest):.1f} %))")

thes-fin:	wrote 89 records (68 train, 21 test (23.6 %))
thes-swe:	wrote 54 records (37 train, 17 test (31.5 %))
thes-eng:	wrote 56 records (43 train, 13 test (23.2 %))
docthes-fin:	wrote 54 records (45 train, 9 test (16.7 %))
docthes-swe:	wrote 25 records (20 train, 5 test (20.0 %))
docthes-eng:	wrote 99 records (84 train, 15 test (15.2 %))
serial-fin:	wrote 74 records (57 train, 17 test (23.0 %))
serial-swe:	wrote 59 records (45 train, 14 test (23.7 %))
serial-eng:	wrote 66 records (49 train, 17 test (25.8 %))
mono-fin:	wrote 77 records (60 train, 17 test (22.1 %))
mono-swe:	wrote 23 records (15 train, 8 test (34.8 %))
mono-eng:	wrote 50 records (37 train, 13 test (26.0 %))


In [6]:
# Generate statistics about the use of metadata fields

from collections import defaultdict, Counter
import datetime
import glob
from statistics import mean
from tabulate import tabulate


langstat_table = tabulate(langstat, headers='keys', tablefmt='github')
repostat_table = tabulate(repostat, headers='keys', tablefmt='github')
coarstat_table = tabulate(coarstat, headers='keys', tablefmt='github')

metadata_files = glob.glob("../metadata/*.jsonl")

field_counts = defaultdict(Counter)  # key1: doctype, key2: field
doc_counts = Counter()
field_types = {}  # key: field, val: S (single) or M (multiple)
field_nvals = defaultdict(list)  # key: field, val: list of number of values

for mdfile in sorted(metadata_files):
    with open(mdfile) as infile:
        for line in infile:
            rec = json.loads(line)
            for fld in rec:
                field_counts[rec['doctype']][fld] += 1
                field_types[fld] = 'M' if isinstance(rec[fld], list) else 'S'
                if field_types[fld] == 'M':
                    field_nvals[fld].append(len(rec[fld]))
            doc_counts[rec['doctype']] += 1

# keep the first (non-dc) columns intact, sort the remaining (dc) fields
all_fields = list(field_types.keys())
all_fields = all_fields[:6] + sorted(all_fields[6:])

data = []

def format_value(val):
    if not val:
        return '-'
    return "{:.0%}".format(val)

for fld in all_fields:
    
    if field_types[fld] == 'M':
        mean_val = mean(field_nvals[fld] or [0])
        max_val = max(field_nvals[fld] or [0])
        ftype = f"M ({mean_val:.1f}/{max_val})"
    else:
        ftype = 'S'
    
    
    row = {
        'Field': fld,
        'Type': ftype
    }
    for doctype in SHEET_NAMES.keys():
        row[doctype] = format_value(field_counts[doctype][fld] / doc_counts[doctype])
    data.append(row)

# Create a Pandas DataFrame from the data
df = pd.DataFrame(data)

# Convert DataFrame to markdown table
field_table = tabulate(df, headers='keys', tablefmt='github', showindex=False)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

statfile = f"""# Statistics about metadata

Automatically generated {timestamp}

Type is either:
 * S: single-value
 * M: multi-value, number of values given as (mean/max)
 
Percentages represent the coverage of a field in a subset. 100% coverage means the field is always present.

## Document counts by language and document type

{langstat_table}

## Document counts by repository and document type

{repostat_table}

## Document counts by COAR resource type and document type

{coarstat_table}

## Metadata coverage by document type

{field_table}
"""

print(statfile)

with open('../statistics.md', 'w') as outf:
    print(statfile, file=outf)

# Statistics about metadata

Automatically generated 2023-09-07 08:52:53

Type is either:
 * S: single-value
 * M: multi-value, number of values given as (mean/max)
 
Percentages represent the coverage of a field in a subset. 100% coverage means the field is always present.

## Document counts by language and document type

| language/iso   |   thes |   docthes |   serial |   mono |   TOTAL |
|----------------|--------|-----------|----------|--------|---------|
| fin            |     89 |        54 |       74 |     77 |     294 |
| eng            |     56 |        99 |       66 |     50 |     271 |
| swe            |     54 |        25 |       59 |     23 |     161 |
| TOTAL          |    199 |       178 |      199 |    150 |     726 |

## Document counts by repository and document type

| Repository   |   thes |   docthes |   serial |   mono |   TOTAL |
|--------------|--------|-----------|----------|--------|---------|
| Theseus      |     91 |         0 |       95 |     45 |     231